### Tautology checker

Develop a function that decides if simple logical propositions are always true. Such propositions are called
tautologies.

Consider a language of propositions built up from basic values (False,True) and variables (A, B, ..., Z)
using negation (¬), conjunction (^), implication (->), and parentheses. For example, the following are all
propositions:

p1= A ∧ ¬A

p2= (A ∧ B) ⇒ A

p3= A ⇒ (A ∧ B)

p4= (A ∧ (A ⇒ B)) ⇒ B

The meaning of the logical operators can be defined using truth tables, which give the resulting value
for each combination of argument values:

| A | B |\|| A -> B|      |A  |\| | A ^ ¬A |
|:- | -:|-:| :-|          |:- | -: |-:      |
|F | F |\|| T|            | F  |\| | T|
|F | T |\|| T|            | T  |\| | T|
|T | F |\|| F|            |    |   | |
|T | T |\|| T|            |    |   | |




### Resolution method:

1) Define a type to represent propositions

2) Define function **tautology** that input a proposition p and return True/False when p is/isn't a tautology

3) The reasoning method is: extract all the variables in the proposition defining function **vars**, for example is p= A -> B it should return (A,B), remove duplicates defining a general function **rmdups** that removes duplicates from any list, create the rows of the truth table as a substitute list, for example, [('A',False),('B',False)], [('A', False), ('B', True)], [('A',True), ('B',False)], [('A', True), ('B', True)] defining function **substs**. Define first a function **bools** which, for example, will return all eight lists of three logical values:

? bools 3 returns

[[False, False, False],
[False, False, True],
[False, True, False],
[False, True, True],
[True, False, False],
[True, False, True],
[True, True, False],
[True, True, True]]


4) Define a type for defining substitute lists as list of tuples (Char,Bool). Define function **find** that return the logical value of a variable in a substitute list. Define function **eval** that given a proposition and a substitute list returns the evaluation of the proposition according with the values of its variables. 


In [1]:
-- Tautology checker

-- A proposition can be a const value True o False, a variable named by a char, or can be generated from
-- logic operators not, and and imply (we omit or in our definition)
data Prop = Const Bool| Var Char | Not Prop | And Prop Prop| Imply Prop Prop deriving Show

p1 :: Prop
p1 = And (Var 'A') (Not (Var 'A'))

p2 :: Prop
p2 = Imply (And (Var 'A') (Var 'B')) (Var 'A')

p3 :: Prop
p3 = Imply (Var 'A') (And (Var 'A') (Var 'B'))

p4 :: Prop
p4 = Imply (And (Var 'A') (Imply (Var 'A') (Var 'B'))) (Var 'B')

-- To evaluate a proposition to a logical value we need to know the value of each of its variables. 
-- We declare a substitution as a lookup table that associates variables names to logical values
-- Ex. the substitution [('A',False), ('B',True)] assigns A=False and B=True
type Assoc k v = [(k,v)]
type Subst = Assoc Char Bool

find :: Eq k => k -> Assoc k v -> v
find k t = head [v | (k',v) <- t, k == k']

find 'A' [('A',False), ('B',True)]

-- A function that evaluates a proposition given a substitution for its variables can now be defined by
-- pattern matching on the five possible forms that the proposition can have:
eval :: Subst -> Prop -> Bool
eval _ (Const b)      = b
eval s (Var x)        = find x s
eval s (Not p)        = not (eval s p)
eval s (And p q)      = eval s p && eval s q
eval s (Imply p q)    = eval s p <= eval s q
-- the logical implication operator is implemented by the <= ordering on logical values.
-- False < True and the only False in the implication A -> B is when (A=True, B=False)
-- Logical implication A-> B is false only when (A=True and B=Fasle), so it's equivalent to say A <= B
True <= False
False <= True 

{-To decide if a proposition is a tautology, we will consider all possible substitutions for the variables
that it contains. First of all, we define a function that returns a list of all the variables in a proposition.
For example, vars p2 = [’A’,’B’,’A’]. Note that this function does not remove duplicates, which will
be done separately later on.
-}
vars :: Prop -> [Char]
vars (Const _)    = []
vars (Var x)      = [x]
vars (Not p)      = vars p
vars (And p q)    = vars p ++ vars q
vars (Imply p q)  = vars p ++ vars q

vars p2

-- Function to remove duplicates from a list
rmdups :: Eq a => [a] -> [a]
rmdups []   =  []
rmdups (x:xs) = x : filter (/= x) (rmdups xs)

{- The key to generating substitutions is producing lists of logical values of a given length. Hence we seek
to define a function bools :: Int -> [[Bool]] which, for example, will return all eight lists of three
logical values:
> bools 3
[[False, False, False],
[False, False, True],
[False, True, False],
[False, True, True],
[True, False, False],
[True, False, True],
[True, True, False],
[True, True, True]]
-}

bools :: Int -> [[Bool]]
bools 0   = [[]]
bools n   = map (False:) bss ++ map (True:) bss
     where bss = bools (n-1)

bools 1
bools 2
bools 3

{- Function that generate all possible possible substitutions for a proposition by 
extracting its variables, removing duplicates from this list, 
generating all possible lists of logical values for this many variables, and then 
zipping the list of variables with each of the resulting lists:
-}

substs  :: Prop -> [Subst]
substs p = map (zip vs) (bools (length vs))
           where vs = rmdups (vars p)

substs p2

{- Finally, we define a function that decides if a proposition is a tautology, by simply checking if it
evaluates to True for all possible substitutions:
-}

isTaut :: Prop -> Bool
isTaut p = and [eval s p | s <- substs p]

isTaut p1
isTaut p2
isTaut p3
isTaut p4
           
          

False

False

True

"ABA"

[[False],[True]]

...

[[('A',False),('B',False)],[('A',False),('B',True)],[('A',True),('B',False)],[('A',True),('B',True)]]

False

True

False

True

In [9]:
-- Redefine the show methods of Prop
data Prop' = Const' Bool| Var' Char | Not' Prop' | And' Prop' Prop'| Imply' Prop' Prop' 

p1' = And' (Var' 'A') (Not' (Var' 'A'))

p2' = Imply' (And' (Var' 'A') (Var' 'B')) (Var' 'A')

p3' = Imply' (Var' 'A') (And' (Var' 'A') (Var' 'B'))

p4' = Imply' (And' (Var' 'A') (Imply' (Var' 'A') (Var' 'B'))) (Var' 'B')

instance Show Prop' where
    show (Const' a) = show a
    show (Var' a) = [a] -- if you put (show a) return 'A'
    show (Not' p) =  "¬" ++ show p 
    show (And' p1 p2) = show p1 ++ " ^ " ++ show p2
    show (Imply' p1 p2) = show p1 ++ " -> " ++ show p2 

{- with parentheses
    show (Const' a) = show a
    show (Var' a) = [a] -- if you put (show a) return 'A'
    show (Not' p) = "~(" ++ show p ++ ")"
    show (And' p1 p2) = "(" ++ show p1 ++ ") ^ (" ++ show p2 ++ ")"
    show (Imply' p1 p2) = "(" ++ show p1 ++ ") -> (" ++ show p2 ++ ")"
-}


p1'
p2'
p3'
p4'

-- Without redefining show it would be:
p1
p2 
p3 
p4

A ^ ¬A

A ^ B -> A

A -> A ^ B

A ^ A -> B -> B

And (Var 'A') (Not (Var 'A'))

Imply (And (Var 'A') (Var 'B')) (Var 'A')

Imply (Var 'A') (And (Var 'A') (Var 'B'))

Imply (And (Var 'A') (Imply (Var 'A') (Var 'B'))) (Var 'B')

### Binary string transmitter

Consider the problem of simulating the transmission of a string of characters in low-level form as a list of binary digits.

For example, the binary number 1101 can be understood as follows:
1101 = (2^3 * 1) +(2^2 * 1) +(2^1 * 0) +(2^0 * 1) = (8 * 1) +(4 * 1) +(2 * 0) +(1 * 1).
That is, 1101 represents the sum of the products of the weights 8, 4, 2, 1 with the bits 1, 1, 0, 1, which
evaluates to the integer 13.

To simplify the definition of certain functions, we assume for the remainder of this example that binary
numbers are written in reverse order to normal. For example, 1101 would now be written as 1011, with
successive bits as we move to the right increasing in weight by a factor of two:
1011 = (1 * 1) +(2 * 0) +(4 * 1) +(8 * 1)



### Resolution method:

We have to convert each char in the string into its Unicode number using function ord (we need Data.Char library). Convert the number into a bit, defining function **int2bin**. Truncates or extends a binary number to make it precisely eight bits, defining function **make8**. Encodes a string of characters as a list of bits by
converting each character into a Unicode number, converting each such number into an eight-bit binary number, and
concatenating all these numbers together to produce a list of bits, defining function **encode**.
For example, ? encode "abc" will return [1,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,1,1,0,0,0,1,1,0] (ord a = 97, int2bin 97 = [1,0,0,0,0,1,1], make8 (int2bin 97) = [1,0,0,0,0,1,1,0,0], repeat the same with 'b' and 'c').

To decode a list of bits produced using encode define function **encode**. First define a function **chop8** that chops such a list up into eight-bit binary numbers. For example, chop8 [1,0,1,0,1,0,1,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1] returns
[[1,0,1,0,1,0,1,0],[1,1,1,1,1,1,1,1],[0,0,0,0,0,0,0,0],[1,1,1]] Convert each eight-bit binary number into an integer, defining **bin2int**. Transform the Unicode number into a char using function char and concatenate the result.

Finally, define the function **transmit** as *transmit = decode . channel . encode*, where function **channel** is just the **id** function. 

Start by defining a type Bit as a synonym of integer. Then define functions **bin2int** and **int2bin**. Afterward define functions **make8**, **endode**, **chop8**, **decode**, **channel** and **transmit**.

? transmit "higher-order functions are easy" returns "higher-order functions are easy"

In [19]:
--Binary string transmitter
import Data.Char

-- we declare a type for bits as a synonym for the type of integers:
type Bit = Int

-- A binary number, represented as a list of bits, can be converted into an integer by simply evaluating 
--the appropriate weighted sum:

bin2int' :: [Bit] -> Int
bin2int' bits = sum [w*b | (w,b) <- zip weights bits]
     where weights = iterate (*2) 1
      
{-The higher-order library function iterate produces an infinite list by applying a function an increasing
number of times to a value:
iterate f x = [x, f x, f (f x), f (f (f x)), ...]     

Hence the expression iterate (*2) 1 in the definition of bin2int produces the list of weights
[1,2,4,8,...], which is then used to compute the weighted sum by means of a list comprehension. For
example:
> bin2int [1,0,1,1]
13
-}

bin2int' [1,0,1,1] 

{-There is a simpler way to define bin2int with the aid of some algebra
(1 * a) + (2 * b) + (4 * c) + (8 * d)
can be restructured as follows:
= a + (2 * b) + (4 * c) + (8 * d)
= a + 2 * (b + (2 * c) + (4 * d))
= a + 2 * (b + 2 * (c + (2 * d)))
= a + 2 * (b + 2 * (c + 2 * (d + 2 * 0)))
-}

bin2int :: [Bit] -> Int
bin2int = foldr (\x y -> x + 2*y) 0

bin2int [1,0,1,1]

-- Funtion that converts an integer into a binary number
int2bin :: Int -> [Bit]
int2bin 0 = []
int2bin n = n `mod` 2 : int2bin (n `div`  2)

int2bin 3
mod 3 2
div 3 2
1: int2bin 1

int2bin 13
mod 13 2
div 13 2
1: int2bin 6

int2bin 6
0: int2bin 3

-- We will ensure that all our binary numbers have the same length, in this case eight bits, by using a
-- function make8 that truncates or extends a binary number as appropriate to make it precisely eight bits:

make8 :: [Bit] -> [Bit]
--make8 bits = take 8 (bits ++ repeat 0) --not curried version
make8 = take 8 . (++ repeat 0)


{-The library function repeat :: a -> [a] produces an infinite list of copies of a value, but lazy
evaluation ensures that only as many elements as required by the context will actually be produced. For
example:> make8 [1,0,1,1]
[1,0,1,1,0,0,0,0]
-}

make8 [1,0,1,1]

 {-We can now define a function that encodes a string of characters as a list of bits by converting each
character into a Unicode number, converting each such number into an eight-bit binary number, and
concatenating each of these numbers together to produce a list of bits. Using the higher-order functions
map and composition, this conversion can be implemented as follows:
-}
encode :: String -> [Bit]
encode = concat . map (make8 . int2bin . ord)

-- :t concat
-- concat [[1], [0,1,1], [3,4]] result [1,0,1,2,3,4]

ord 'a'
int2bin (ord 'a')

encode "abc"
-- encode "a"
-- encode "b"
-- encode "c"

{-To decode a list of bits produced using encode, we first define a function chop8 that chops such a list
up into eight-bit binary numbers:-}

chop8 :: [Bit] -> [[Bit]]
chop8 []     = []
chop8 bits   = take 8 bits : chop8 (drop 8 bits)

chop8 [1,0,1,0,1,0,1,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1]
-- result [[1,0,1,0,1,0,1,0],[1,1,1,1,1,1,1,1],[0,0,0,0,0,0,0,0],[1,1,1]]
-- drop 3 [1..10] result [4,5,6,7,8,9,10]
-- take 3 [1..10] result [1,2,3] 

{-It is now easy to define a function that decodes a list of bits as a string of characters by chopping the list
up, and converting each resulting binary number into a Unicode number and then a character:-}
decode :: [Bit] -> String
decode = map (chr . bin2int) . chop8

:t chr -- (chr) :: Int -> Char

encode "hello" --result [0,0,0,1,0,1,1,0,1,0,1,0,0,1,1,0,0,0,1,1,0,1,1,0,0,0,1,1,0,1,1,0,1,1,1,1,0,1,1,0]
chop8 [0,0,0,1,0,1,1,0,1,0,1,0,0,1,1,0,0,0,1,1,0,1,1,0,0,0,1,1,0,1,1,0,1,1,1,1,0,1,1,0]
-- result [[0,0,0,1,0,1,1,0],[1,0,1,0,0,1,1,0],[0,0,1,1,0,1,1,0],[0,0,1,1,0,1,1,0],[1,1,1,1,0,1,1,0]]
(chr . bin2int) [0,0,0,1,0,1,1,0]
(chr . bin2int) [1,0,1,0,0,1,1,0]

decode [0,0,0,1,0,1,1,0,1,0,1,0,0,1,1,0,0,0,1,1,0,1,1,0,0,0,1,1,0,1,1,0,1,1,1,1,0,1,1,0]

{-Finally, we define a function transmit that simulates the transmission of a string of characters as a list
of bits, using a perfect communication channel that we model using the identity function:-}

transmit :: String -> String
transmit = decode . channel . encode

channel :: [Bit] -> [Bit]
channel = id

transmit "higher-order functions are easy"


13

13

[1,1]

1

1

...

[[0,0,0,1,0,1,1,0],[1,0,1,0,0,1,1,0],[0,0,1,1,0,1,1,0],[0,0,1,1,0,1,1,0],[1,1,1,1,0,1,1,0]]

'h'

'e'

"hello"

"higher-order functions are easy"

### Voting algorithms

We consider two different algorithms for deciding the winner in an election: the simple first past the post system, and the more refined alternative vote system.

First past the post. 
In this system, each person has one vote, and the candidate with the largest number of votes is declared
the winner. For example, if we define
votes :: [String]
votes = ["Red", "Blue", "Green", "Blue", "Blue", "Red"]
then candidate "Green" has one vote, "Red" has two votes, while "Blue" has three votes and is hence the
winner.

Alternative vote. In this voting system, each person can vote for as many or as few candidates as they wish, listing them in preference order on their ballot (1st choice, 2nd choice, and so on).

### Resolution method:

#### First past the post

votes = ["Red", "Blue", "Green", "Blue", "Blue", "Red"]

1) Define a function **count** that counts the number of times that a given value occurs in a list. For example, 

? count "Red" votes returns 2

2) Define a function that removes duplicate values from a list

3) Define a function **result** that returns the result of a first-past-the-post election in increasing order of the number of votes received. For example, 

? result votes   returns [(1,"Green"), (2,"Red"), (3,"Blue")]

4) Define function **winner**


#### Alternative vote

ballots = [["Red", "Green"],
           ["Blue"],
           ["Green", "Red", "Blue"],
           ["Blue", "Green", "Red"],
           ["Green"]]
           
The first ballot has "Red" as 1st choice and "Green" as 2nd, while the second has "Blue" as the only
choice, and so on. Now let us consider how the winner is decided for this example. First of all, "Red"
has the smallest number of 1st-choice votes (just one), and is therefore eliminated:

[["Green"],
["Blue"],
["Green", "Blue"],
["Blue", "Green"],
["Green"]]

Within these revised ballots, candidate "Blue" now has the smallest number of 1st-choice votes (just
 two), and is therefore also eliminated:
 
[["Green"],
[],
["Green"],
["Green"],
["Green"]]

After removing the second ballot, which is now empty, "Green" is the only remaining candidate and is
hence the winner.

1) Using filter and map, define functions **rmempty** that remove empty ballots, and **elim** that eliminate a given candidate from each ballot. Define such functions in a general manner rather than just for strings. 

2) In turn, using the function result from the previous section, define a function **ranks** that ranks the 1st-choice candidates in each ballot in increasing order of the number of such votes that were received

? rank ballots  returns ["Red", "Blue", "Green"]

3) Finally, define a recursive function that implements the alternative vote algorithm **winner'** 
           




In [3]:
{-In this system, each person has one vote, and the candidate with the largest number of votes is declared
the winner. For example, if we define-}
import Data.List

votes :: [String]
votes = ["Red", "Blue", "Green", "Blue", "Blue", "Red"]


{-then candidate "Green" has one vote, "Red" has two votes, while "Blue" has three votes and is hence the
winner. Rather than making our implementation specific to candidate names represented as strings, we
exploit the class system of Haskell to define our functions in a more general manner.
First of all, we define a function that counts the number of times that a given value occurs in a list, for
any type whose values can be compared for equality. This function could be defined using recursion, but a
simpler definition is possible using higher-order functions by selecting all elements from the list that are
equal to the target value, and taking the length of the resulting list:
-}

count :: Eq a => a -> [a] -> Int
count x = length . filter (== x)

-- count "Red" votes  result 2

-- The higher-order function filter can also be used to define a function that removes duplicate values from a list:

rmdups :: Eq a => [a] -> [a]
rmdups []       = []
rmdups (x:xs)   = x : filter (/= x) (rmdups xs)

-- filter (/= 1) [1,2,3,1,1] result [2,3]

-- rmdups votes   result ["Red", "Blue", "Green"]

{- The functions count and rmdups can then be combined using a list comprehension to define a function
that returns the result of a first-past-the-post election in increasing order of the number of votes received:-}
result :: Ord a => [a] -> [(Int,a)]
result vs = sort [(count v vs, v) | v <- rmdups vs]

-- > result votes   result [(1,"Green"), (2,"Red"), (3,"Blue")]

{-The sorting function sort :: Ord a => [a] -> [a] used above is provided in the library Data.List .
Note that because pairs are ordered lexicographically, candidates with the same number of votes are
returned in order of the candidate name by result. Finally, the winner of an election can now be obtained
simply by selecting the second component of the last result:-}

winner :: Ord a => [a] -> a
winner = snd . last . result

{-Alternative vote.
In this voting system, each person can vote for as many or as few candidates as they wish, listing them in
preference order on their ballot (1st choice, 2nd choice, and so on). To decide the winner, any empty
ballots are first removed, then the candidate with the smallest number of 1st-choice votes is eliminated
from the ballots, and same process is repeated until only one candidate remains, who is then declared the
winner. For example, if we define-}

ballots :: [[String]]
ballots = [["Red", "Green"],
           ["Blue"],
           ["Green", "Red", "Blue"],
           ["Blue", "Green", "Red"],
           ["Green"]]

{-then the first ballot has "Red" as 1st choice and "Green" as 2nd, while the second has "Blue" as the only
choice, and so on. Now let us consider how the winner is decided for this example. First of all, "Red"
has the smallest number of 1st-choice votes (just one), and is therefore eliminated:
[["Green"],
["Blue"],
["Green", "Blue"],
["Blue", "Green"],
["Green"]]

Within these revised ballots, candidate "Blue" now has the smallest number of 1st-choice votes (just
 two), and is therefore also eliminated:
[["Green"],
[],
["Green"],
["Green"],
["Green"]]
After removing the second ballot, which is now empty, "Green" is the only remaining candidate and is
hence the winner.-}

{-Using filter and map , it is easy to define functions that remove empty ballots, and eliminate a given
candidate from each ballot:-}

rmempty :: Eq a => [[a]] -> [[a]]
rmempty = filter (/= [])

elim :: Eq a => a -> [[a]] -> [[a]]
elim x = map (filter (/= x))

{-As before, we define such functions in a general manner rather than just for strings. In turn, using the
function result from the previous section, we can define a function that ranks the 1st-choice candidates
in each ballot in increasing order of the number of such votes that were received:-}

rank :: Ord a => [[a]] -> [a]
rank = map snd . result . map head

-- map head ballots  return ["Red","Blue","Green","Blue","Green"]
-- result ["Red","Blue","Green","Blue","Green"] return [(1,"Red"),(2,"Blue"),(2,"Green")]
-- map snd [(1,"Red"),(2,"Blue"),(2,"Green")]

-- rank ballots  result ["Red", "Blue", "Green"]

{-Finally, it is now straightforward to define a recursive function that implements the alternative vote
algorithm, as follows:-}

winner' :: Ord a => [[a]] -> a
winner' bs = case rank (rmempty bs) of
                 [c] -> c
                 (c:cs) -> winner' (elim c bs)
                 
rank ballots
r1 = elim "Red" ballots
r1

rank (rmempty r1)
r2 = elim "Blue" r1
r2

rank (rmempty r2)
             
{-That is, we first remove empty ballots, then rank the remaining 1st-choice candidates in increasing order
of votes. If only one such candidate remains, they are the winner, otherwise we eliminate the candidate
with the smallest number of 1st-choice votes and repeat the process. For example:
> winner’ ballots "Green"-}

winner' ballots

["Red","Blue","Green"]

[["Green"],["Blue"],["Green","Blue"],["Blue","Green"],["Green"]]

["Blue","Green"]

[["Green"],[],["Green"],["Green"],["Green"]]

["Green"]

"Green"